In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
%%capture
import torch
import json
import pandas as pd
from datasets import Dataset
import warnings
warnings.filterwarnings("ignore")

In [5]:
from unsloth import FastLanguageModel
max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    loftq_config = None,
)

Unsloth 2024.12.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import json
import pandas as pd
from datasets import Dataset

json_file_path = '/content/PFE_dataset1.json'

with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["text"]
    outputs = examples["code"]

    texts = []
    for instruction, output in zip(instructions, outputs):
        text = instruction + "\n" + output + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

if isinstance(data, list):
    data_df = pd.DataFrame(data)
else:
    data_df = data

dataset = Dataset.from_pandas(data_df, split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/775 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=16,
        warmup_steps=20,
        num_train_epochs=15,
        learning_rate=3e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=2,
        optim="adamw_8bit",
        lr_scheduler_type="cosine",
        weight_decay=0.01,
        seed=0,
        output_dir="outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map (num_proc=2):   0%|          | 0/775 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 775 | Num Epochs = 15
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 16
\        /    Total batch size = 256 | Total steps = 45
 "-____-"     Number of trainable parameters = 59,768,832


Step,Training Loss
2,1.966600
4,2.828700
6,1.814000
8,2.370700


In [ ]:
def generate_code(request):
    FastLanguageModel.for_inference(model)
    inputs = tokenizer(
    [
        prompt.format(
           request,
            "",
        )
    ], return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=1024)


prompt = """
Task Description:
{}

PySpark Code:
"""


request = """
first, Join customers and orders, then filter for orders from 2023"""


generate_code(request)

In [ ]:
# Merge to 16bit
if True: model.save_pretrained_merged("PFE_2024_V2", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("ehiri/PFE_2024_V2", tokenizer, save_method = "merged_16bit", token = "xxxxxx")

# Merge to 4bit
if False: model.save_pretrained_merged("pyspark_code_generator_PFE_2024_V2", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("ehiri/pyspark_code_generator_PFE_2024_V2", tokenizer, save_method = "merged_4bit", token = "xxxx")



In [ ]:
if False: model.push_to_hub_merged(
    "ehiri/pyspark_code_generator_PFE_2024_V2",
    tokenizer,
    save_method = "merged_16bit",
    token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")